<a href="https://colab.research.google.com/github/marco-siino/DA-ESWA/blob/main/code/SVM_HSS_not_augmented_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Text preprocessing worth the time: A comparative survey on the impact of common techniques on NLP model performances. 
- - - 
SVM ON FAKE NEWS SPREADERS DS EXPERIMENTS NOTEBOOK 
- - -
Support Vector Machine on Fake News Spreaders Dataset.
Code by M. Siino. 

From the paper: "Text preprocessing worth the time: A comparative survey on the impact of common techniques on NLP model performances." by M.Siino et al.



## Importing modules.

In [1]:
import matplotlib.pyplot as plt
import os
import random
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from google.colab import files
from io import open
from numpy.random import seed
import numpy as np
from pathlib import Path
from sklearn import svm

os.environ['TF_CUDNN_DETERMINISTIC']='true'
os.environ['TF_DETERMINISTIC_OPS']='true'

## Importing DS and extract in current working directory.

In [2]:
# Url obtained starting from this: https://drive.google.com/file/d/19ZcqEv88euKB71HfAWjTGN3uCKp2qsfP/ and forcing export=download.
urlTrainingSet = "https://github.com/marco-siino/DA-ESWA/blob/main/data/hss/hss-training-original.zip?raw=true"
urlTestSet="https://github.com/marco-siino/DA-ESWA/blob/main/data/hss/hss-test-cleaned.zip?raw=true"

training_set = tf.keras.utils.get_file("pan21-author-profiling-training-2021-03-14.zip", urlTrainingSet,
                                    extract=True, archive_format='zip',cache_dir='.',
                                    cache_subdir='')
test_set = tf.keras.utils.get_file("pan21-author-profiling-test-without-gold-augmented.zip", urlTestSet,
                                    extract=True, archive_format='zip',cache_dir='.',
                                    cache_subdir='')

training_set_dir = os.path.join(os.path.dirname(training_set), 'pan21-author-profiling-training-2021-03-14')
test_set_dir = os.path.join(os.path.dirname(test_set), 'pan21-author-profiling-test-without-gold-augmented')

print(training_set)
print(training_set_dir)

!ls -A

653272/653272 [==============================] - 0s 0us/step
./pan21-author-profiling-training-2021-03-14.zip
./pan21-author-profiling-training-2021-03-14
.config
pan21-author-profiling-test-without-gold-augmented
pan21-author-profiling-test-without-gold-augmented.zip
pan21-author-profiling-training-2021-03-14
pan21-author-profiling-training-2021-03-14.zip
sample_data


## Build folders hierarchy to use Keras folders preprocessing function.

In [3]:
### Training Folders. ###

# First level directory.
if not os.path.exists('train_dir_en'):
    os.makedirs('train_dir_en')

# Class labels directory.
if not os.path.exists('train_dir_en/0'):
    os.makedirs('train_dir_en/0')
if not os.path.exists('train_dir_en/1'):
    os.makedirs('train_dir_en/1')

# Make Py variables.
train_dir='train_dir_'

## Test Folders. ##
# First level directory.
if not os.path.exists('test_dir_en'):
    os.makedirs('test_dir_en')

# Class labels directory.
if not os.path.exists('test_dir_en/0'):
    os.makedirs('test_dir_en/0')
if not os.path.exists('test_dir_en/1'):
    os.makedirs('test_dir_en/1')

# Make Py variables.
test_dir='test_dir_'

!ls -A

.config
pan21-author-profiling-test-without-gold-augmented
pan21-author-profiling-test-without-gold-augmented.zip
pan21-author-profiling-training-2021-03-14
pan21-author-profiling-training-2021-03-14.zip
sample_data
test_dir_en
train_dir_en


## Set language and directory paths.


In [5]:
# Set en and es ground truth file path for train_dir. We haven't a ground truth file for the test set.
language='en'

truth_file_training_dir_en=training_set_dir+'/'+language+'/'
truth_file_training_path_en = truth_file_training_dir_en+'truth.txt'

truth_file_test_dir=test_set_dir
truth_file_test_path_en = truth_file_test_dir+'/'+'truth'+'.txt'

## Read truth.txt to organize training and test dataset folders.



In [6]:
# Open the file truth.txt with read only permit.
f = open(truth_file_training_path_en, "r")
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_training_dir_en+fNameXml):
      os.rename(truth_file_training_dir_en+fNameXml, './train_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

# Open the file truth.txt with read only permit.
f = open(truth_file_test_path_en, "r")
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_test_dir+'/'+language+'/'+fNameXml):
      os.rename(truth_file_test_dir+'/'+language+'/'+fNameXml, './test_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

## Generate full training set.



In [7]:
# Generate full randomized training set.
batch_size=1

en_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir+language, 
    batch_size=batch_size,
    shuffle=False
    )

en_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    test_dir+language, 
    batch_size=batch_size,
    shuffle=False
    )

train_ds=en_train_ds.shuffle(300,seed=1, reshuffle_each_iteration=False)
test_ds=en_test_ds.shuffle(200,seed=1, reshuffle_each_iteration=False)

train_ds_size=len(train_ds)
test_ds_size=len(test_ds)

Found 200 files belonging to 2 classes.
Found 100 files belonging to 2 classes.


## Functions to pre-process source text. (A detailed discussion on our paper)

In [8]:
# Do-Nothing preprocessing function.
def DON(input_data):
  tag_open_CDATA_removed = tf.strings.regex_replace(input_data, '<\!\[CDATA\[', ' ')
  tag_closed_CDATA_removed = tf.strings.regex_replace(tag_open_CDATA_removed,'\]{1,}>', ' ')
  tag_author_lang_en_removed = tf.strings.regex_replace(tag_closed_CDATA_removed,'<author lang="en">', ' ')
  tag_closed_author_removed = tf.strings.regex_replace(tag_author_lang_en_removed,'</author>', ' ')
  tag_open_documents_removed = tf.strings.regex_replace(tag_closed_author_removed,'<documents>\n(\t){0,2}', '')
  output_data = tf.strings.regex_replace(tag_open_documents_removed,'</documents>\n(\t){0,2}', ' ')
  return output_data

## Get the length of the longest sample in training set. Then adapt text.



In [9]:
def preprocess_and_adapt_ts(preprocessing_function,training_set):
  # Set a very large sequence length to find the longest sample in the training set.
  sequence_length = 10000
  vectorize_layer = TextVectorization(
      standardize=preprocessing_function,
      output_mode='int',
      output_sequence_length=sequence_length)

  train_text = training_set.map(lambda x, y: x)
  vectorize_layer.adapt(train_text)
  #vectorize_layer.get_vocabulary()

  model = tf.keras.models.Sequential()
  model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
  model.add(vectorize_layer)

  longest_sample_length=1

  for element in training_set:
    authorDocument=element[0]
    label=element[1]
    
    #print("Sample considered is: ", authorDocument[0].numpy())
    #print("Preprocessed: ", str(custom_standardization(authorDocument[0].numpy())))
    #print("And has label: ", label[0].numpy())

    # Count the number of zeros from the last non-zero token to the end of the sample. 
    # Shortest tokenized sample has less zeros than others.
    out=model(authorDocument)
    token_nr_index=sequence_length-1
    current_sample_zeros_counter=0
    while out.numpy()[0][token_nr_index]==0:
      token_nr_index-=1
      current_sample_zeros_counter+=1

    shortest_padding_length=sequence_length-longest_sample_length
    if current_sample_zeros_counter<shortest_padding_length:
      longest_sample_length=sequence_length-current_sample_zeros_counter

  #print(out.numpy()[0][3229:3400])
  #print(longest_sample_length)

  # After tokenization longest_sample_length covers all the document lenghts in our dataset.
  sequence_length = longest_sample_length

  vectorize_layer = TextVectorization(
      standardize=preprocessing_function,
      output_mode='int',
      output_sequence_length=sequence_length)

  # Finally adapt the vectorize layer.
  train_text = training_set.map(lambda x, y: x)
  vectorize_layer.adapt(train_text)
  return vectorize_layer

## Define a dictionary with -> function_names:prepro_function_caller. And a dictionary to store model results.




In [10]:
model_results = {}
prepro_functions_dict_base = {
    'DON':DON,
    }

# 3 prepro functions = 15 combs...+1 for do_nothing

prepro_functions_dict_comb = {
    # 1. Do nothing 
    'DON': DON,
}

for key in prepro_functions_dict_comb:
  print(key)
  model_results[key]=[]

DON


## Models definition and evaluation.




In [11]:
for key in prepro_functions_dict_comb:
    print("\n\n* * * * EVALUATION USING", key, "AS PREPROCESSING FUNCTION * * * *")

    # Preprocess training set to build a dictionary.
    vectorize_layer = preprocess_and_adapt_ts(prepro_functions_dict_comb[key],train_ds)

    print("\n\n***** FINITO DI PROCESSARE E ADATTARE IL TRAINING SET, INIZIA LA SIMULAZIONE *******")
    # Print a raw and a preprocessed sample.
    for element in train_ds:
      authorDocument=element[0]
      label=element[1]
      
      print("Sample considered is: ", authorDocument[0])
      print("Preprocessed: ", str(prepro_functions_dict_comb[key](authorDocument[0].numpy())))
      break
    
    # # # - - - - - MODELS DEFINITION AND EVALUATION - - - - - # # #

    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
    model.add(vectorize_layer)
    
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

    # --- SVM SECTION START --- #

    training_labels=[]
    training_samples=[]

    max_features=len(vectorize_layer.get_vocabulary()) + 1

    for element in train_ds:
      authorDocument=element[0]
      label=element[1]
      
      #print("Sample considered is: ", authorDocument[0])
      #print("Preprocessed: ", str(custom_standardization(authorDocument[0].numpy())))
      #print("And has label: ", label[0].numpy())
      
      text_vect_layer_model = tf.keras.Model(inputs=model.input,
                                          outputs=model.layers[0].output)
      text_vect_out = text_vect_layer_model(authorDocument)

      training_labels.append(label[0].numpy())
      current_sample=np.zeros(max_features)
      for current_token in text_vect_out[0][:].numpy():
        #print(current_token,end=' ')
        #print(vectorize_layer.get_vocabulary()[current_token])
        current_sample[current_token]+=1
      training_samples.append(current_sample)
      #break

    training_labels=np.array(training_labels)
    training_samples=np.array(training_samples)
    #print("\nLE LABELS DEI CAMPIONI DI TRAINING SONO:")
    #print(training_labels)
    #print("\nI SAMPLE DI TRAINING DOPO LA TEXT VECTORIZATION SONO:")
    #print(training_samples)

    test_labels=[]
    test_samples=[]

    for element in test_ds:
      authorDocument=element[0]
      label=element[1]
      
      text_vect_layer_model = tf.keras.Model(inputs=model.input,
                                          outputs=model.layers[0].output)
      text_vect_out = text_vect_layer_model(authorDocument)

      test_labels.append(label[0].numpy())
      current_sample=np.zeros(max_features)
      for current_token in text_vect_out[0][:].numpy():
        current_sample[current_token]+=1
      test_samples.append(current_sample)

    test_labels=np.array(test_labels)
    test_samples=np.array(test_samples)

    SVM = svm.SVC(C=0.5, kernel='linear', gamma='auto')
    SVM.fit(training_samples,training_labels)
    # predict the labels on training set
    #predictions_SVM = SVM.predict(training_samples)
    # Use accuracy_score function to get the accuracy
    #model_results[key]['SVM_train']=SVM.score(training_samples,training_labels)
    #print("SVM Accuracy Score on Training set -> ",model_results[key]['SVM_train'])

    # predict the labels on validation dataset
    predictions_SVM = SVM.predict(test_samples)
    # Use accuracy_score function to get the accuracy
    model_results[key].append(SVM.score(test_samples,test_labels))
    print("SVM Accuracy Score on Test set -> ",model_results[key])

    # --- SVM SECTION END --- #

    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
    
    # # # - - - - - MODEL DEFINITION AND EVALUATION END  - - - - - # # #

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089




* * * * EVALUATION USING DON AS PREPROCESSING FUNCTION * * * *


***** FINITO DI PROCESSARE E ADATTARE IL TRAINING SET, INIZIA LA SIMULAZIONE *******
Sample considered is:  tf.Tensor(b'<author lang="en" class="1">\n\t<documents>\n\t\t<document><![CDATA[#USER# Where was this at?]]></document>\n\t\t<document><![CDATA[RT #USER#: Black man,has a stern warning for BLM, don\'t mess with white people \xf0\x9f\x98\xac #URL#]]></document>\n\t\t<document><![CDATA[#USER# A meeting of insurrectionists...I mean a Biden rally]]></document>\n\t\t<document><![CDATA[#USER# #USER# This is lame. Instead of stringing up curlers, they should be stringing up...ahem]]></document>\n\t\t<document><![CDATA[#USER# This is insane. The citizens better rise up. And this is why you never willingly give up your weapons.]]></document>\n\t\t<document><![CDATA[#USER# I would have wrote: \xe2\x80\x9cMy name is... and I am not a racist, but Al Sharpton is a racist and I will pray for him to do better\xe2\x80\x9d]]></doc

## Now show compact results in a table.

In [ ]:
print(" PREPRO FUNCTION    |  Test Accuracy   |",end = '')

print("\n")
for prepro_func in prepro_functions_dict_comb:
  #print(prepro_func,"\t\t\t",format(round(model_results[prepro_func][0],4),'.4f'),"\t\t",end='')
  result = format(round(model_results[prepro_func][0],4),'.4f')
  print(f'{prepro_func:27}{ result :12}')
  print("\n")
 

 PREPRO FUNCTION    |  Test Accuracy   |

DON                        0.6300      


